In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pubg-finish-placement-prediction/train_V2.csv
/kaggle/input/pubg-finish-placement-prediction/test_V2.csv
/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv


In [2]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/train_V2.csv")
test = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/test_V2.csv")


In [4]:
train.dropna(axis=0, inplace= True)

In [5]:
label_name = 'winPlacePerc'
X_tr_all = train.loc[ : , ['assists','kills','killPoints','DBNOs','killStreaks','boosts','killPlace','revives','heals','matchType']]
train = train.astype({'winPlacePerc': 'int'})
y_tr_all = train[label_name]
X_last_test = test.loc[ : , ['assists','kills','killPoints','DBNOs','killStreaks','boosts','killPlace','revives','heals','matchType']]

In [6]:
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'solo-fpp':11})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'duo-fpp':21})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'solo':13})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'duo':23})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'squad':43})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'squad-fpp':41})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-solo-fpp':110})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-duo-fpp': 210})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-squad-fpp':410})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-solo':130})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-duo':230})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'normal-squad':430})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'crashfpp':10})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'crashtpp':30})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'flarefpp':19})
X_tr_all['matchType'] = X_tr_all['matchType'].replace({'flaretpp':39})
X_tr_all['matchType']

0          41
1          41
2          23
3          41
4          11
           ..
4446961    41
4446962    13
4446963    41
4446964    41
4446965    11
Name: matchType, Length: 4446965, dtype: int64

In [7]:
X_last_test['matchType'] = X_last_test['matchType'].replace({'solo-fpp':11})
X_last_test['matchType'] = X_last_test['matchType'].replace({'duo-fpp':21})
X_last_test['matchType'] = X_last_test['matchType'].replace({'solo':13})
X_last_test['matchType'] = X_last_test['matchType'].replace({'duo':23})
X_last_test['matchType'] = X_last_test['matchType'].replace({'squad':43})
X_last_test['matchType'] = X_last_test['matchType'].replace({'squad-fpp':41})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-solo-fpp':110})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-duo-fpp': 210})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-squad-fpp':410})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-solo':130})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-duo':230})
X_last_test['matchType'] = X_last_test['matchType'].replace({'normal-squad':430})
X_last_test['matchType'] = X_last_test['matchType'].replace({'crashfpp':10})
X_last_test['matchType'] = X_last_test['matchType'].replace({'crashtpp':30})
X_last_test['matchType'] = X_last_test['matchType'].replace({'flarefpp':19})
X_last_test['matchType'] = X_last_test['matchType'].replace({'flaretpp':39})
X_last_test['matchType']

0          41
1          21
2          41
3          21
4          41
           ..
1934169    41
1934170    41
1934171    41
1934172    21
1934173    41
Name: matchType, Length: 1934174, dtype: int64

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X_tr_all,y_tr_all,random_state=33)

In [9]:
model = LGBMRegressor()
model = model.fit(X_train,y_train)
pred = model.predict(X_test)
pred

array([ 1.42760608e-06,  7.96583716e-03,  1.86932697e-01, ...,
        3.38283970e-02, -2.23609848e-04, -6.85767522e-05])

In [10]:
pred_last = model.predict(X_last_test)
pred_last

array([0.00020143, 0.14909014, 0.0120348 , ..., 0.05242977, 0.01639123,
       0.00025455])

In [11]:
sample_submission = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv")
sample_submission.head()


,Id,winPlacePerc
0,9329eb41e215eb,1
1,639bd0dcd7bda8,1
2,63d5c8ef8dfe91,1
3,cf5b81422591d1,1
4,ee6a295187ba21,1


In [12]:
sample_submission["winPlacePerc"] = pred_last
sample_submission.to_csv("submission.csv", index=False)